## Model: Model_Meta-Llama-3-8B-Instruct

In [20]:
!pip install --upgrade --no-cache-dir \
  transformers \
  accelerate \
  sentence-transformers \
  langchain \
  pinecone-client \
  datasets \
  bitsandbytes \
  langchain-community \
  langchain-pinecone \
  pinecone \
  auto-gptq

!pip uninstall -y optimum
!pip install --upgrade --no-cache-dir "optimum[onnxruntime]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 252.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.6/424.6 kB 375.8 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.0
    Uninstalling transformers-4.53.0:
      Successfully uninstalled transformers-4.53.0


In [1]:
import os
from langchain_community.vectorstores import Pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema.document import Document
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec
import pandas as pd

# Load .env manually or define here
from google.colab import userdata
PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')
HF_TOKEN=userdata.get('HF_TOKEN')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("/content/drive/MyDrive/my_project/processed_cleaned_chunks.csv")

docs = [
    Document(
        page_content=row['text'],
        metadata={"chunk_id": row['chunk_id'], "video_id": row['video_id']}
    )
    for _, row in df.iterrows()
]

In [4]:
pinecone = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'youtube-transcripts'
if index_name not in [i['name'] for i in pinecone.list_indexes()]:
    pinecone.create_index(
        name=index_name,
        dimension=384,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

vectors = []
for doc in docs:
    embedding = embed_model.encode(doc.page_content).tolist()
    vectors.append({
        'id': doc.metadata['chunk_id'],
        'values': embedding,
        'metadata': doc.metadata
    })

index = pinecone.Index(index_name)
index.upsert(vectors)

{'upserted_count': 328}

In [5]:
from huggingface_hub import login

login(token=HF_TOKEN)

In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    trust_remote_code=True
)

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Mistral-7B-Instruct-v0.2-GPTQ were not used when initializing MistralForCausalLM: ['model.

In [15]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone as PineconeClient
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Reconnect to your Pinecone index
#import pinecone
pc = PineconeClient(api_key=PINECONE_API_KEY)
#pinecone.init(api_key=PINECONE_API_KEY, environment="us-east-1")
index_name ="youtube-transcripts"
index = pc.Index(index_name)

vectorstore = PineconeVectorStore(index=index, embedding=embeddings, text_key="text")
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [16]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)

Device set to use cuda:0
/tmp/ipython-input-16-1581924493.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)


In [17]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [22]:
from langchain_core.output_parsers import StrOutputParser
import logging
logging.getLogger("langchain_pinecone.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "How do I create a catalog item in ServiceNow?"
response = qa_chain(query)
# Clean up boilerplate LLM response content
raw_output = response["result"]

# Heuristically trim everything before "Helpful Answer:"
if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()

print(cleaned)

To create a Catalog Item in ServiceNow, you would typically follow these steps:
1. Log in to your instance as an admin or a user with appropriate permissions.
2. Navigate to the "Catalog" application in the left-side menu.
3. Click on the "Items" tab within the "Catalog" application.
4. Click the "New" button to create a new catalog item record.
5. Fill out the required fields such as Name, Description, Category, etc.
6. Add any additional details such as Price, Vendor Information, etc.
7. Set up any dependencies, such as requiring certain other items to be ordered before this one.
8. Save the record.

However, please note that creating a catalog item involves more than just filling out a form. It also requires understanding the various options and configurations associated with the catalog item, which may require additional expertise or training. If you're unsure about how to create a catalog item or have specific questions, it may be best to consult with your ServiceNow administrator

In [24]:
logging.getLogger("langchain_pinecone.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "What is ITSM in ServiceNow"
response = qa_chain(query)
# Clean up boilerplate LLM response content
raw_output = response["result"]

# Heuristically trim everything before "Helpful Answer:"
if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(query)
print(cleaned)

What is ITSM in ServiceNow
In ServiceNow, ITSM stands for IT Service Management. It refers to the set of processes, practices, and tools that organizations use to manage and deliver IT services effectively and efficiently. These services can range from incident management, problem management, change management, knowledge management, and more. The goal of ITSM is to ensure that IT services align with business needs, meet agreed-upon SLAs (Service Level Agreements), and continuously improve through data-driven insights and analytics. Mike Brickner, a Service Offering Manager at ServiceNow, mentioned how ITSM plays a crucial role in reducing downtime for manufacturing customers by providing valuable insights and improving communication between IT and the factory floor teams.


In [25]:
logging.getLogger("langchain_pinecone.vectorstores").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
query = "What are different AI concepts used in ServiceNow?"
response = qa_chain(query)
# Clean up boilerplate LLM response content
raw_output = response["result"]

# Heuristically trim everything before "Helpful Answer:"
if "Answer:" in raw_output:
    cleaned = raw_output.split("Answer:")[-1].strip()
else:
    cleaned = raw_output.strip()
print(query)
print(cleaned)

What are different AI concepts used in ServiceNow?
ServiceNow is deeply invested in the three pillars of AI that work together to create a comprehensive solution. These pillars include:

1. Data Fabric: This aspect of AI in ServiceNow involves combining data from various sources such as different discourses, infrastructure, and external data sources. The data is then integrated into a single platform, providing a unified view for analysis.

2. AI Agent: The AI agent is responsible for analyzing the data provided by the Data Fabric and creating intelligent actions based on that analysis. It can be programmed to learn from historical data and identify patterns, trends, and anomalies. For instance, in manufacturing industries, the AI agent can help identify potential issues leading to downtime before they occur, allowing factories to minimize losses.

3. Assisted and Autonomous Workflow: ServiceNow also offers assisted and autonomous workflows, which enable collaboration between humans an